In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.spatial.distance import cdist
from scipy import stats

#To get better visual of the confusion matrix:
def plot_confusion_matrix(cm, classes,
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):
    #Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix\\\\")
    #else:
        #print('Confusion matrix, without normalization\\\\')
    
#     print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

use_pca = True

#carichiamo i dati 
df = pd.read_csv("exoplanet.csv")
df.info()
df = df.replace('CONFIRMED', 0)
df = df.replace('CANDIDATE', 1)
df = df.replace('FALSE POSITIVE', 2)
df = df.fillna(0)

df.to_csv("Exoplanet_mod.csv")
df

In [ ]:
#normalize feature with MinMaxScaler after it we need to fit the data
MinMaxScaler = preprocessing.MinMaxScaler()

y_data = pd.read_csv('Exoplanet_mod.csv', usecols= ['koi_disposition'])
x_data = pd.read_csv('Exoplanet_mod.csv', usecols= ['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec','koi_period','koi_period_err1','koi_period_err2','koi_time0bk','koi_time0bk_err1','koi_time0bk_err2','koi_impact','koi_impact_err1','koi_impact_err2','koi_duration','koi_duration_err1','koi_duration_err2','koi_depth','koi_depth_err1','koi_depth_err2','koi_prad','koi_prad_err1','koi_prad_err2','koi_teq','koi_teq_err1','koi_teq_err2','koi_insol','koi_insol_err1','koi_insol_err2','koi_model_snr','koi_tce_plnt_num','koi_steff','koi_steff_err1','koi_steff_err2','koi_slogg','koi_slogg_err1','koi_slogg_err2','koi_srad','koi_srad_err1','koi_srad_err2','ra','dec','koi_kepmag'])

#take the 80% of data for train and 20% for test
x_train = x_data[:7650]
y_train = y_data[:7650]

x_test = x_data[7650:]
y_test = y_data[7650:]

x_train = x_train.values
x_test = x_test.values

print("Classes train : ", y_train.shape)
print("Features train : ", x_train.shape)

print("Classes test :", y_test.shape)
print("Features train : ", x_test.shape)

x_train = MinMaxScaler.fit_transform(x_train)
x_test = MinMaxScaler.fit_transform(x_test)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA().fit(x_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()
    
cum_var = np.cumsum(pca.explained_variance_ratio_)

XT = pca.transform(x_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 1. Inizializzo i parametri

"""
Metrics intended for real-valued vector spaces:

“euclidean” EuclideanDistance sqrt(sum((x - y)^2))

“manhattan” ManhattanDistance sum(|x - y|)

“chebyshev” ChebyshevDistance max(|x - y|)

“minkowski” MinkowskiDistance p sum(|x - y|^p)^(1/p)

“wminkowski” WMinkowskiDistance p, w sum(|w * (x - y)|^p)^(1/p)

“seuclidean” SEuclideanDistance V sqrt(sum((x - y)^2 / V))

“mahalanobis” MahalanobisDistance V or VI sqrt((x - y)' V^-1 (x - y))
"""

 #These are the predicted output values

In [ ]:
from sklearn.metrics import accuracy_score
ks = [5,7,11,13,15,17]
funs = ["euclidean", "manhattan","chebyshev","minkowski"]#,"seuclidean","mahalanobis"]

V = np.cov(x_train)
classes = ['0','1','2']

def f1(x):
    return '%1.2f' % x
def f2(x):
    return '%i' % x

train_accuracy = np.empty(len(ks))
test_accuracy = np.empty(len(ks))
train_scores, test_scores = list(), list()

for fun in funs:
    i=0
    for k in ks:
        if fun=="minkowski":
            knn_clf=KNeighborsClassifier(n_neighbors=k, weights='distance', metric=fun, p=8)
        elif fun == "manhattan":
            knn_clf=KNeighborsClassifier(n_neighbors=k, weights='distance', metric=fun, p=1)
        else:
            knn_clf=KNeighborsClassifier(n_neighbors=k, weights='distance', metric=fun)
        
        print("#################", fun ,"[K=%i]  #################" % k)
        knn_clf.fit(x_train,y_train)
        predict=knn_clf.predict(x_test)

        classification_metrics = metrics.classification_report(y_test, np.round(predict), target_names=classes)
        cm_dict = metrics.classification_report(y_test, np.round(predict), target_names=classes, output_dict=True)
        #print("\nParameters (k,metric):",k,fun)
        print(classification_metrics)
        print(fun[0].capitalize()+fun[1:]+"\\\\")
        #print(df)

        confusion_matrix= metrics.confusion_matrix(y_test, predict)

        plot_confusion_matrix(confusion_matrix, classes)
        plt.show()
        plot_confusion_matrix(confusion_matrix, classes, normalize=True)
        plt.show()

        #test new graph to show accuracy 
        test_accuracy[i] = knn_clf.score(x_test, y_test)
        knn_clf.fit(x_test,y_test)
        train_accuracy[i] = knn_clf.score(x_train, y_train)
        i = i+1

    plt.plot(ks, test_accuracy, label = 'Testing dataset Accuracy')
    plt.plot(ks, train_accuracy, label = 'Training dataset Accuracy')
    
    plt.legend()
    plt.xlabel('n_neighbors')
    plt.ylabel('Accuracy')
    plt.show()

        
        #tp, fp, tn, fn = perf_measure(y_test.values, predict)

        #TSS = (tp/(tp+fn) - (fp/(fp+tn)))
        #accuracy = (tp+tn)/(tp+fp+fn+tn)


        #print('[tp fp tn fn] = ', tp, fp, tn, fn)
        #print('Accuracy: ', accuracy)
        #print('TSS: ', TSS)
